In [1]:
! pip install fuzzywuzzy
! pip install python-Levenshtein

In [12]:
import polars as pl
from polars import col, when, lit
from datetime import date, datetime, timedelta
from fuzzywuzzy import fuzz

In [15]:
from colorama import Fore, Back, Style

def colored_string(text, color="normal", bg_color="normal", style="normal") -> str:
    """
    returns colored text using Colorama

    Parameters:
        text (str): The text to be printed.
        color (str): The text color. Default is white.
        bg_color (str): The background color. Default is black.
        style (str): The text style. Default is normal.

    Returns:
        None
    """

    # if bg_color or color is None, replace with normal
    color = color or 'normal'
    bg_color = bg_color or 'normal'
    style = style or 'normal'
    
    # Map string inputs to Colorama constants
    color_mapping = {
        "black": Fore.BLACK,
        "red": Fore.RED,
        "green": Fore.GREEN,
        "yellow": Fore.YELLOW,
        "blue": Fore.BLUE,
        "magenta": Fore.MAGENTA,
        "cyan": Fore.CYAN,
        "white": Fore.WHITE,
        "lightgreen": Fore.LIGHTGREEN_EX,
        "lightred": Fore.LIGHTRED_EX,
        "lightblue": Fore.LIGHTBLUE_EX,
        "normal": Fore.RESET,
    }
    bg_color_mapping = {
        "black": Back.BLACK,
        "red": Back.RED,
        "green": Back.GREEN,
        "yellow": Back.YELLOW,
        "blue": Back.BLUE,
        "magenta": Back.MAGENTA,
        "cyan": Back.CYAN,
        "white": Back.WHITE,
        "normal": Back.RESET,
    }
    style_mapping = {
        "normal": Style.NORMAL,
        "bright": Style.BRIGHT,
        "dim": Style.DIM,
    }

    # Get the Colorama constants for color, background color, and style
    selected_color = color_mapping.get(color.lower(), Fore.RESET)
    selected_bg_color = bg_color_mapping.get(bg_color.lower(), Back.RESET)
    selected_style = style_mapping.get(style.lower(), Style.NORMAL)

    # Construct the colored text
    if color == "normal" and bg_color == "normal":
        return f"{text}"
    else:
        return f"{selected_style}{selected_bg_color}{selected_color}{text}{Style.RESET_ALL}"


def colored_print(text, color="normal", bg_color="normal", style="normal"):
    """
    Wrapper function for print that prints colored text using Colorama.

    Parameters:
        text (str): The text to be printed.
        color (str): The text color. Default is white.
        bg_color (str): The background color. Default is black.
        style (str): The text style. Default is normal.

    Returns:
        None
    """
    if color == "normal" and bg_color == "normal":
        print(text)
    else:
        print(colored_string(text, color, bg_color, style))

In [6]:
df_alior = pl.read_excel('20241005_Alior.xlsx')
df_taxxo = pl.read_excel('20241005_TAXXO.xlsx')

In [7]:
# condition the columns
df_taxxo = df_taxxo.with_columns(
    indeks_taxxo=col('Lp'),
    data_wystawienia=col('data_wystawienia').dt.date(),
    data_zaplaty=col('data_zaplaty').dt.date(),
)

df_alior = df_alior.with_columns(
    indeks_alior=col('Lp'),
    data_transakcji=col('data_transakcji').str.to_date()
)


In [16]:
colored_print('hello', color='orange')

hello


In [8]:
df_taxxo

Lp,opis_taxxo,data_wystawienia,Kontrahent,kwota,Tagi,Nr rejestracyjny,data_zaplaty,indeks_taxxo
i64,str,date,str,f64,str,str,date,i64
1,"""FV-2024/01/000095""",2024-01-01,"""LIVEKID SPÓŁKA Z OGRANICZONĄ O…",41.57,"""OGÓLNE""","""KONTO_DOTACJA""",2024-01-03,1
2,"""01/2311/00010213 z dnia 04.12.…",2024-01-01,"""PGE OBRÓT SPÓŁKA AKCYJNA""",214.89,"""NIERUCHOMOŚĆ""","""KONTO_DOTACJA""",2024-01-16,2
3,"""19/01/2024""",2024-01-02,"""GASTROPULS.PL SPÓŁKA Z OGRANIC…",342.84,"""OGÓLNE""","""KONTO_DOTACJA""",2024-01-18,3
4,"""3571448/37/2024/F""",2024-01-02,"""PGNIG OBRÓT DETALICZNY SPÓŁKA …",1915.71,"""NIERUCHOMOŚĆ""","""KONTO_DOTACJA""",2024-01-16,4
5,"""2/01/2024""",2024-01-02,"""ANDIX Joanna Mirecka""",246.0,"""OGÓLNE""","""KONTO_DOTACJA""",2024-01-25,5
…,…,…,…,…,…,…,…,…
375,"""2/10/2024""",2024-10-01,"""ANDIX Joanna Mirecka""",246.0,"""OGÓLNE""","""KONTO_GLOWNE""",2024-10-04,375
376,"""01/10/2024""",2024-10-01,"""STANISŁAWA JELEŃ ACTIV RENT""",14183.13,"""NIERUCHOMOŚĆ""","""KONTO_GLOWNE""",2024-10-03,376
377,"""050/BOT/10/2024""",2024-10-01,"""PRZEDSIĘBIORSTWO OSZCZĘDZANIA …",36.9,"""NIERUCHOMOŚĆ""","""KONTO_GLOWNE""",2024-10-04,377


In [9]:
df_alior

Lp,data_transakcji,nadawca,odbiorca,opis,kwota,indeks_alior
i64,date,str,str,str,f64,i64
1,2024-10-04,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""PRZEDSIĘBIORSTWO OSZCZĘDZANIA …","""050/BOT/10/2024""",-36.9,1
2,2024-10-04,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""KANCELARIA RACHUNKOWO-PODATKOW…","""F/063/09/2024""",-457.56,2
3,2024-10-04,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""""","""POBRANIE OPŁATY/PROWIZJI - Rea…",-1.0,3
4,2024-10-04,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""ANDIX Joanna Mirecka""","""Zlecenie stale parking Fiszera""",-246.0,4
5,2024-10-03,"""LIVEKID SP. Z O.O.UL.ENERGETYC…","""SMOCZA DOLINA Sławomir Jeleń""","""ZWROT OD LIVEKID""",489.33,5
…,…,…,…,…,…,…
589,2024-01-10,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""Przelew własny""",2050.0,589
590,2024-01-05,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""Pożyczka 2024""",-1000.0,590
591,2024-01-05,"""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""SŁAWOMIR JELEŃ SMOCZA DOLINA""","""Przelew własny""",2970.0,591


In [23]:
threshold_opis = 0.30      # opis similarity threshold
threshold_kwota = 1.5      # kwota difference threshold in PLN
threshold_date = 3         # days between taxxo and alior transfer

# here we will make changes
taxxo_verified = []

# iterate over taxxo entries
for r in df_taxxo.iter_rows(named=True):
    found = False
    taxxo_date = r['data_zaplaty']
    taxxo_kwota = r['kwota']
    taxxo_opis = r['opis_taxxo']
    taxxo_index = r['Lp']

    
    # find matching alior entries
    # for x in df_alior.filter(col('data_transakcji')==taxxo_date).iter_rows(named=True):
    for x in df_alior.iter_rows(named=True):
        alior_date = x['data_transakcji']
        alior_kwota = x['kwota']
        alior_opis = x['opis']
        alior_index = x['Lp']

        # match conditions
        try:
            # compute similarity
            similarity_opis = fuzz.ratio(taxxo_opis, alior_opis)
    
            # value differences
            diff_kwota = abs(taxxo_kwota + alior_kwota)
            diff_date = abs((taxxo_date - alior_date).days)

            # run conditions
            if ( diff_kwota <= threshold_kwota  # kwota difference within threshold
            and similarity_opis/100 > threshold_opis  # compare description similarity
            and diff_date <= threshold_date):  # compare date with threshold
                
                # print message
                msg = f'[taxxo-{taxxo_index}:alior-{alior_index}] kwoty: {taxxo_kwota:,}/{taxxo_date} ? {alior_kwota:,}/{alior_date} -> {similarity_opis}%'
                if diff_kwota > 0: msg += colored_string(f' ({diff_kwota:.1f})', color='red')
                if similarity_opis >= 85 and abs(taxxo_kwota + alior_kwota) == 0:
                    print(msg)
                elif similarity_opis >= 75 and abs(taxxo_kwota + alior_kwota) == 0:
                    colored_print(msg, color='yellow', style='bright')
                elif similarity_opis < 75 or abs(taxxo_kwota + alior_kwota) > 0:
                    colored_print(msg, color='yellow', style='dim')
    
                # add to verified list
                taxxo_verified.append({
                    'indeks_taxxo': taxxo_index,
                    'verification_found': True,
                    'verification_indeks_alior': alior_index,
                    'verification_opis': similarity_opis/100,
                    'verification_kwota_difference': abs(taxxo_kwota + alior_kwota),
                    'verification_date_difference': abs((taxxo_date - alior_date).days),
                })
    
                # stop
                break
        except:
            pass

df_taxxo_verified = pl.DataFrame(taxxo_verified)

[taxxo-11:alior-561] kwoty: 324.6/2024-02-11 ? -324.6/2024-02-11 -> 100%
[taxxo-14:alior-570] kwoty: 414.0/2024-02-06 ? -414.0/2024-02-06 -> 100%
[taxxo-29:alior-582] kwoty: 60.27/2024-01-31 ? -60.0/2024-01-30 -> 44% (0.3)
[taxxo-35:alior-568] kwoty: 548.96/2024-02-08 ? -548.96/2024-02-08 -> 74%
[taxxo-36:alior-547] kwoty: 408.97/2024-02-28 ? -408.97/2024-02-28 -> 62%
[taxxo-40:alior-557] kwoty: 150.0/2024-02-14 ? -150.0/2024-02-14 -> 90%
[taxxo-41:alior-567] kwoty: 602.7/2024-02-08 ? -602.7/2024-02-08 -> 100%
[taxxo-46:alior-562] kwoty: 457.56/2024-02-11 ? -457.56/2024-02-11 -> 100%
[taxxo-52:alior-566] kwoty: 1,190.0/2024-02-08 ? -1,190.0/2024-02-08 -> 100%
[taxxo-53:alior-565] kwoty: 2,730.0/2024-02-08 ? -2,730.0/2024-02-08 -> 77%
[taxxo-55:alior-552] kwoty: 184.5/2024-02-20 ? -185.5/2024-02-20 -> 74% (1.0)
[taxxo-57:alior-390] kwoty: 14,183.13/2024-04-29 ? -14,183.13/2024-04-29 -> 71%
[taxxo-59:alior-564] kwoty: 261.0/2024-02-10 ? -261.0/2024-02-09 -> 100%
[taxxo-63:alior-555] kwot

In [ ]:
# merge
df_taxxo_verified

In [ ]:
df_taxxo_joined = df_taxxo.join(other=df_taxxo_verified, on='Lp', how='left')
df_taxxo_joined.write_excel('20241005_TAXXO_verified.xlsx')
df_taxxo.write_excel('20241005_TAXXO.xlsx')
df_alior.write_excel('20241005_Alior.xlsx')